In [1]:
import pandas as pd

project_path = '/home/wjunneng/Ubuntu/2019-CCF-Passenger-Car-Segment-Sales-Forecast'

# train_sales_data
train_sales_data_path = project_path + '/data/original/train_sales_data.csv'
# train_search_data
train_search_data_path = project_path + '/data/original/train_search_data.csv'
# train_user_reply_data
train_user_reply_data = project_path + '/data/original/train_user_reply_data.csv'
# evaluation_public
evaluation_public = project_path + '/data/original/evaluation_public.csv'
# submit_example.csv
submit_example = project_path + '/data/original/submit_example.csv'

train_sales_data = pd.read_csv(train_sales_data_path)
# train_search_data = pd.read_csv(train_search_data_path)
# train_user_reply_data = pd.read_csv(train_user_reply_data)
evaluation_public = pd.read_csv(evaluation_public)

data = pd.concat([train_sales_data, evaluation_public], axis=0, ignore_index=True)
data['bodyType'] = data['model'].map(train_sales_data.drop_duplicates('model').set_index('model')['bodyType'])
data.drop(labels='forecastVolum', axis=1, inplace=True)
data.drop(labels='id', axis=1, inplace=True)
print(data.info())

data['date'] = data['regYear'].astype(str) + '-' + data['regMonth'].astype(str)
data.drop(labels='regYear', axis=1, inplace=True)
data.drop(labels='regMonth', axis=1, inplace=True)
print(data.info())

import datetime

data['date'] = data['date'].apply(lambda x: datetime.datetime(year=int(x.split('-')[0]), month=int(x.split('-')[1]), day=1))

print(data.head())

data['bodyType'] = data['bodyType'].apply(lambda x: 1 if x == 'Sedan' else x)
data['bodyType'] = data['bodyType'].apply(lambda x: 2 if x == 'SUV' else x)
data['bodyType'] = data['bodyType'].apply(lambda x: 3 if x == 'MPV' else x)
data['bodyType'] = data['bodyType'].apply(lambda x: 4 if x == 'Hatchback' else x)

model = dict(zip(data['model'].unique(), range(1, data['model'].nunique())))

print(model)

data['model'] = data['model'].map(model)
print(data.head())

data.drop(labels='province', axis=1, inplace=True)
data['adcode'] = data['adcode'].apply(lambda x: int(str(x)[:2]))

X_train = pd.DataFrame(columns=data.columns)
# X_valid = pd.DataFrame(columns=data.columns)
X_test = pd.DataFrame(columns=data.columns)

for key, value in data.groupby(by=['bodyType', 'model', 'adcode']):
    value.sort_values(by='date', inplace=True)
    value.reset_index(inplace=True)
    X_train = pd.concat([X_train, value.loc[:23, :]], axis=0, ignore_index=True)
    # X_valid = pd.concat([X_valid, value.loc[20:23, :]], axis=0, ignore_index=True)
    X_test = pd.concat([X_test, value.loc[24:, :]], axis=0, ignore_index=True)
    

/home/wjunneng/Python/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:21: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

/home/wjunneng/Python/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/wjunneng/Python/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:60: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the curre

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36960 entries, 0 to 36959
Data columns (total 7 columns):
adcode         36960 non-null int64
bodyType       36960 non-null object
model          36960 non-null object
province       36960 non-null object
regMonth       36960 non-null int64
regYear        36960 non-null int64
salesVolume    31680 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 2.0+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36960 entries, 0 to 36959
Data columns (total 6 columns):
adcode         36960 non-null int64
bodyType       36960 non-null object
model          36960 non-null object
province       36960 non-null object
salesVolume    31680 non-null float64
date           36960 non-null object
dtypes: float64(1), int64(1), object(4)
memory usage: 1.7+ MB
None
   adcode bodyType             model province  salesVolume       date
0  310000      SUV  3c974920a76ac9c1       上海        292.0 2016-01-01
1  530000      SUV  3c974920a76

In [2]:
path = '/home/wjunneng/Ubuntu/2019-CCF-Passenger-Car-Segment-Sales-Forecast/demo/lstm/data'

X_train = X_train[['date', 'adcode', 'bodyType', 'model', 'salesVolume']]
# X_valid = X_valid[['date', 'adcode', 'bodyType', 'model', 'salesVolume']]
X_test = X_test[['date', 'adcode', 'bodyType', 'model', 'salesVolume']]

X_train.to_csv(path_or_buf=path+'/X_train.csv', index=None, encoding='utf-8')
# X_valid.to_csv(path_or_buf=path+'/X_valid.csv', index=None, encoding='utf-8')
X_test.to_csv(path_or_buf=path+'/X_test.csv', index=None, encoding='utf-8')


In [138]:
from sklearn.preprocessing import MinMaxScaler

X_train = X_train.set_index('date')
X_valid = X_valid.set_index('date')
X_test = X_test.set_index('date')


sc = MinMaxScaler(feature_range=(0,1))
X_train = sc.fit_transform(X_train.iloc[:,:].values)
X_valid = sc.fit_transform(X_valid.iloc[:,:].values)
X_test = sc.fit_transform(X_test.iloc[:,:].values)

X_train, y_train = X_train[:,:-1], X_train[:,-1]
X_val, y_val = X_valid[:,:-1], X_valid[:,-1]
X_test, y_test = X_test[:,:-1], X_test[:,-1]

X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
# Initialising RNN
regressor = Sequential()
# Adding the first LSTM layer and some Dropout regularization
# Dropout regularization is added to avoid overfitting

regressor.add(LSTM(units = 10, return_sequences = True, activation = 'relu', input_shape = (X_train.shape[1], 4)))
regressor.add(Dropout(0.5))
# adding a second LSTM layer and some dropout regularization
regressor.add(LSTM(units = 10, return_sequences = True, activation = 'relu'))
regressor.add(Dropout(0.5))
# adding a fourth LSTM layer and some dropout regularization
regressor.add(LSTM(units = 10, return_sequences = False, activation = 'relu'))
regressor.add(Dropout(0.5))
# Adding the output layer
#regressor.add(Flatten())
regressor.add(Dense(units=1, activation = 'sigmoid'))
# Compiling the RNN
regressor.compile(optimizer='adam', 
                  loss='mean_squared_error', 
                  metrics=['crossentropy'])
# Fitting the RNN to the training set
history = regressor.fit(X_train, 
              y_train, 
              epochs = 10, 
              batch_size = 1024, 
              validation_data = (X_val, y_val),
              verbose = 1)



/home/wjunneng/Python/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/preprocessing/data.py:355: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
/home/wjunneng/Python/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/preprocessing/data.py:356: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


Train on 25960 samples, validate on 5192 samples
Epoch 1/10
25960/25960 [==============================] - 3s 110us/step - loss: 0.2119 - ce: 0.6866 - val_loss: 0.1933 - val_ce: 0.6792
Epoch 2/10
25960/25960 [==============================] - 0s 9us/step - loss: 0.2043 - ce: 0.6715 - val_loss: 0.1859 - val_ce: 0.6645
Epoch 3/10
25960/25960 [==============================] - 0s 9us/step - loss: 0.1962 - ce: 0.6552 - val_loss: 0.1776 - val_ce: 0.6477
Epoch 4/10
25960/25960 [==============================] - 0s 9us/step - loss: 0.1865 - ce: 0.6358 - val_loss: 0.1668 - val_ce: 0.6261
Epoch 5/10
25960/25960 [==============================] - 0s 9us/step - loss: 0.1723 - ce: 0.6069 - val_loss: 0.1482 - val_ce: 0.5884
Epoch 6/10
25960/25960 [==============================] - 0s 9us/step - loss: 0.1464 - ce: 0.5512 - val_loss: 0.1063 - val_ce: 0.5010
Epoch 7/10
25960/25960 [==============================] - 0s 9us/step - loss: 0.1107 - ce: 0.4655 - val_loss: 0.0418 - val_ce: 0.3414
Epoch 8/10


In [139]:
import numpy as np
predicted_sales = regressor.predict(X_test)
predicted_sales


array([[2.7303183e-01],
       [2.0852444e-01],
       [1.3154912e-01],
       ...,
       [7.1525574e-07],
       [2.0861626e-07],
       [0.0000000e+00]], dtype=float32)

In [140]:
# Reshape X_test for inverse scaling
result = X_test.reshape((X_test.shape[0], X_test.shape[2]))
print(result.shape, predicted_sales.shape)
# Concatenate in the same order. In our example, values of weekly sales should be in the end. Hence result[:,:] followed by predicted_sales
predicted_weekly_sales = np.concatenate((result, predicted_sales),axis=1)
predicted_weekly_sales.shape


(5192, 4) (5192, 1)


(5192, 5)

In [141]:
predicted_weekly_sales = sc.inverse_transform(predicted_weekly_sales)


In [142]:
predicted_weekly_sales = predicted_weekly_sales[:,4:5]


In [143]:
predicted_weekly_sales

array([[nan],
       [nan],
       [nan],
       ...,
       [nan],
       [nan],
       [nan]])